In [1]:
import pandas as pd
import glob
import os

In [2]:
data_dir = '../data'
genders = ['Men', 'Women']
csv_lists = {}
# Generate a list of pathnames to relevant CSVs
for gender in genders:
    csv_lists[gender] = glob.glob(os.path.join(data_dir, 'FSA_DS_{}*'.format(gender)))

In [61]:
# Instantiate a dict of empty DataFrames to collect all our data
dfs = {gender: pd.DataFrame() for gender in genders}

for gender in genders:
    # Give the master DataFrame index a meaningful name
    dfs[gender].index.name = 'Rider ID'
    
    for csv_path in csv_lists[gender]:
        # Read in CSV
        df = pd.read_csv(csv_path)
        # Calculate two columns based on filename
        df['Gender'] = gender
        year = int(csv_path[-8:-4])
        df['Year'] = year
        # Clean up identified issues in Rider Name column
        df['Rider Name'] = df['Rider Name'].str.strip()
        df['Rider Name'] = df['Rider Name'].str.replace('  ', ' ')
        df['Rider Name'] = df['Rider Name'].str.replace('\\', '')

        # Identify correct age table (birth year and UCI category aren't included in the downloadable CSVs)
        age_path = glob.glob(os.path.join(data_dir, 'age_{}_{}*'.format(gender, year)))[0]
        age = pd.read_csv(age_path, sep = '\t', encoding = 'utf-8')
        # Remove trailing spaces from data and column names
        for col in age.columns:
            if age[col].dtype == 'object':
                age[col] = age[col].str.strip()
        age.columns = age.columns.str.strip()

        if gender == 'Men':
            # Harmonize this table's column names with the CSV's
            age = age.rename(columns = {'Rider': 'Rider Name'})
            # Join birth year and UCI category onto main table
            df = df.merge(age[['Country', 'UCI Team', 'Rider Name', 'UCI Cat', 'Born']], 
                          how = 'left', on = ['Country', 'UCI Team', 'Rider Name'])
            # Append the current year to columns that will change from year to year
            old_cols = ['UCI Team', 'Price', 'UCI Cat']
        else: # The women's table has fewer and differently named columns
            age = age.rename(columns = {'Rider': 'Rider Name', 'Trade Team': 'UCI Team'})
            df = df.merge(age[['Country', 'UCI Team', 'Rider Name', 'Born']], 
                          how = 'left', on = ['Country', 'UCI Team', 'Rider Name'])
            old_cols = ['UCI Team', 'Price']

        new_cols = {col: '{} {}'.format(col, year) for col in old_cols}
        df = df.rename(columns = new_cols)
        # Calculate the riders' age in this year
        df['Age {}'.format(year)] = df['Year'] - df['Born']

        # Drop unnecessary columns
        df = df.drop(columns = ['Rank', 'Score {}'.format(year - 1), 'Year'])
        # Set a useful index
        df = df.set_index('Rider ID')
        
        # Prepare to join new columns to old rows...
        to_join = df.loc[df.index.isin(dfs[gender].index), ~df.columns.isin(dfs[gender].columns)]
        # ...and to append new rows to old columns...
        to_append = df.loc[~df.index.isin(dfs[gender].index), :]
        # ...then do both!
        dfs[gender] = dfs[gender].join(to_join).append(to_append)
        
    # Export the processed data to CSV
    dfs[gender].to_csv(os.path.join(data_dir, gender + '.csv'))

,Country,UCI Team 2012,Rider Name,Price 2012,Score 2012,Gender,Born,Age 2012,UCI Team 2013,Price 2013,...,Score 2018,Age 2018,UCI Team 2019,Price 2019,Score 2019,Age 2019,UCI Team 2020,Price 2020,Score 2020,Age 2020
Rider ID,,,,,,,,,,,,,,,,,,,,,
2030,GBR,XYZ,Jessica Allen,1.0,0.0,Women,1989.0,23.0,FUT,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3647,AUS,NaN,Jessica Allen,NaN,NaN,Women,1993.0,NaN,NaN,NaN,...,77.0,25.0,MTS,1.0,24.0,26.0,MTS,1.0,0.0,27.0


In [15]:
years = [int(col[-4:]) for col in dfs['Men'].columns if 'Age ' in col]
years

[2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020]

In [ ]:
year = 2020
years_ago = 0
records_year = year # the year for which we're currently generating records

while year >= min(years):
    construct a '_Nya' string
    select (via df.loc[]) 
    df['Year'] = year



In [29]:
year = 2016

year_vars = ['Age', 'UCI Team', 'UCI Cat', 'Price', 'Score']
cols = ['{} {}'.format(col, year) for col in year_vars]

df = dfs['Men'].loc[~dfs['Men']['Price {}'.format(year)].isna(), cols]

years_ago = 0

df.columns = ['{} {}ya'.format(col, years_ago) for col in year_vars]

df

,Age 0ya,UCI Team 0ya,UCI Cat 0ya,Price 0ya,Score 0ya
Rider ID,,,,,
1,34.0,TNK,WT,24.0,1840.0
2,36.0,MOV,WT,36.0,2233.0
3,35.0,TFS,WT,18.0,1456.0
4,31.0,DDD,WT,20.0,1456.0
6,34.0,BMC,WT,24.0,809.0
...,...,...,...,...,...
3654,31.0,ABB,CONT,1.0,0.0
3654,31.0,ABB,CONT,1.0,0.0
3655,33.0,UKO,CONT,1.0,40.0


In [30]:
year = 2015

cols = ['{} {}'.format(col, year) for col in year_vars]

df2 = dfs['Men'].loc[~dfs['Men']['Price {}'.format(year)].isna(), cols]

years_ago = 1

df2.columns = ['{} {}ya'.format(col, years_ago) for col in year_vars]

df2

,Age 1ya,UCI Team 1ya,UCI Cat 1ya,Price 1ya,Score 1ya
Rider ID,,,,,
1,33.0,TCS,WT,32.0,1899.0
2,35.0,MOV,WT,38.0,3214.0
3,34.0,TFR,WT,24.0,405.0
4,30.0,EQS,WT,22.0,655.0
6,33.0,BMC,WT,28.0,1455.0
...,...,...,...,...,...
3315,32.0,OPM,CONT,1.0,6.0
3317,22.0,TGA,WT,1.0,30.0
3318,21.0,RNR,CONT,1.0,30.0


In [33]:
df3 = df.join(df2, how = 'left')
df3[df3.index.isin(df.index)]

,Age 0ya,UCI Team 0ya,UCI Cat 0ya,Price 0ya,Score 0ya,Age 1ya,UCI Team 1ya,UCI Cat 1ya,Price 1ya,Score 1ya
Rider ID,,,,,,,,,,
1,34.0,TNK,WT,24.0,1840.0,33.0,TCS,WT,32.0,1899.0
2,36.0,MOV,WT,36.0,2233.0,35.0,MOV,WT,38.0,3214.0
3,35.0,TFS,WT,18.0,1456.0,34.0,TFR,WT,24.0,405.0
4,31.0,DDD,WT,20.0,1456.0,30.0,EQS,WT,22.0,655.0
6,34.0,BMC,WT,24.0,809.0,33.0,BMC,WT,28.0,1455.0
...,...,...,...,...,...,...,...,...,...,...
3654,31.0,ABB,CONT,1.0,0.0,NaN,NaN,NaN,NaN,NaN
3654,31.0,ABB,CONT,1.0,0.0,NaN,NaN,NaN,NaN,NaN
3655,33.0,UKO,CONT,1.0,40.0,NaN,NaN,NaN,NaN,NaN


In [35]:
dfs['Men'].loc[3654]

,Country,UCI Team 2010,Rider Name,Price 2010,Score 2010,Gender,UCI Cat 2010,Born,Age 2010,UCI Team 2011,...,UCI Team 2019,Price 2019,Score 2019,UCI Cat 2019,Age 2019,UCI Team 2020,Price 2020,Score 2020,UCI Cat 2020,Age 2020
Rider ID,,,,,,,,,,,,,,,,,,,,,
3654,DEN,NaN,Martin Toft Madsen,NaN,NaN,Men,NaN,1985.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3654,DEN,NaN,Martin Toft Madsen,NaN,NaN,Men,NaN,1985.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
